In [1]:
import sqlite3
database = "database.sqlite"
conn = sqlite3.connect(database)
cursor = conn.cursor()
print("Data opened successfully.")

Data opened successfully.


In [3]:
import pandas as pd
teams = pd.read_sql("""SELECT Match_Id, Match_Winner, Team_Name
                    FROM Match
                    INNER JOIN Team ON
                    Match.Match_Winner == Team.Team_Id""", conn)

In [4]:
teams

,Match_Id,Match_Winner,Team_Name
0,335987,1,Kolkata Knight Riders
1,335988,3,Chennai Super Kings
2,335989,6,Delhi Daredevils
3,335990,2,Royal Challengers Bangalore
4,335991,1,Kolkata Knight Riders
...,...,...,...
569,981016,2,Royal Challengers Bangalore
570,981018,2,Royal Challengers Bangalore
571,981020,11,Sunrisers Hyderabad
572,981022,11,Sunrisers Hyderabad


In [5]:
teams = pd.read_sql("""SELECT Team_Name, COUNT(Match_Winner)
                    FROM Match
                    INNER JOIN Team ON
                    Match.Match_Winner == Team.Team_Id
                    GROUP BY Team_Name""", conn)

In [6]:
teams

,Team_Name,COUNT(Match_Winner)
0,Chennai Super Kings,79
1,Deccan Chargers,29
2,Delhi Daredevils,56
3,Gujarat Lions,9
4,Kings XI Punjab,63
5,Kochi Tuskers Kerala,6
6,Kolkata Knight Riders,68
7,Mumbai Indians,80
8,Pune Warriors,12
9,Rajasthan Royals,63


In [9]:
teams = pd.read_sql("""SELECT Match_Id, Match_Winner, Team_Name
                    FROM Match
                    LEFT JOIN Team ON
                    Match.Match_Winner == Team.Team_Id""", conn)

In [10]:
teams

,Match_Id,Match_Winner,Team_Name
0,335987,1.0,Kolkata Knight Riders
1,335988,3.0,Chennai Super Kings
2,335989,6.0,Delhi Daredevils
3,335990,2.0,Royal Challengers Bangalore
4,335991,1.0,Kolkata Knight Riders
...,...,...,...
572,981016,2.0,Royal Challengers Bangalore
573,981018,2.0,Royal Challengers Bangalore
574,981020,11.0,Sunrisers Hyderabad
575,981022,11.0,Sunrisers Hyderabad


In [11]:
teams = pd.read_sql("""SELECT c.Country_Id, c.Country_Name, ci.City_Name
                    FROM country c
                    CROSS JOIN city ci""", conn)

In [12]:
teams

,Country_Id,Country_Name,City_Name
0,1,India,Bangalore
1,1,India,Chandigarh
2,1,India,Delhi
3,1,India,Mumbai
4,1,India,Kolkata
...,...,...,...
343,12,Netherlands,Raipur
344,12,Netherlands,Ranchi
345,12,Netherlands,Abu Dhabi
346,12,Netherlands,Rajkot


In [13]:
teams = pd.read_sql("""SELECT Player_Name
                    FROM Player
                    UNION
                    SELECT Team_Name
                    FROM Team""", conn)

In [14]:
teams

,Player_Name
0,A Ashish Reddy
1,A Chandila
2,A Chopra
3,A Flintoff
4,A Kumble
...,...
477,Younis Khan
478,YS Chahal
479,Yuvraj Singh
480,YV Takawale


In [17]:
import sqlite3
# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect("ilp_database.db")
cursor = conn.cursor()
# Create Inventory Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Inventory (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT,
        quantity INTEGER,
        price REAL
    );
''')
# Create Logistics Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Logistics (
        logistics_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        shipment_date TEXT,
        status TEXT,
        FOREIGN KEY (product_id) REFERENCES Inventory (product_id)
    );
''')
# Create Product Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Product (
        product_id INTEGER PRIMARY KEY,
        category TEXT,
        supplier TEXT,
        FOREIGN KEY (product_id) REFERENCES Inventory (product_id)
    );
''')

# Commit changes
conn.commit()
print("Tables created successfully!")
# Insert data into Inventory
cursor.executemany('''
    INSERT INTO Inventory (product_id, product_name, quantity, price)
    VALUES (?, ?, ?, ?);
''', [
    (1, 'Laptop', 15, 750.50),
    (2, 'Monitor', 25, 200.00),
    (3, 'Keyboard', 50, 30.00),
    (4, 'Mouse', 40, 25.00),
    (5, 'Tablet', 10, 500.00)
])
# Insert data into Logistics
cursor.executemany('''
    INSERT INTO Logistics (logistics_id, product_id, shipment_date, status)
    VALUES (?, ?, ?, ?);
''', [
    (1, 1, '2024-03-10', 'Shipped'),
    (2, 2, '2024-03-12', 'In Transit'),
    (3, 3, '2024-03-14', 'Delivered'),
    (4, 5, '2024-03-15', 'Pending')
])
# Insert data into Product
cursor.executemany('''
    INSERT INTO Product (product_id, category, supplier)
    VALUES (?, ?, ?);
''', [
    (1, 'Electronics', 'TechCorp'),
    (2, 'Electronics', 'ScreenPro'),
    (3, 'Accessories', 'KeyWorld'),
    (4, 'Accessories', 'MouseMax'),
    (5, 'Electronics', 'TabMakers')
])
# Commit changes
conn.commit()
print("Sample data inserted successfully!")

Tables created successfully!


IntegrityError: UNIQUE constraint failed: Inventory.product_id

In [16]:
query = """
SELECT p.category, COUNT(i.product_id) AS total_products
FROM Product p
JOIN Inventory i ON p.product_id = i.product_id
GROUP BY p.category
HAVING total_products > 1
ORDER BY total_products DESC;
"""

cursor.execute(query)
rows = cursor.fetchall()

print("\nHAVING Clause Results:")
for row in rows:
    print(row)


HAVING Clause Results:
('Electronics', 3)
('Accessories', 2)


In [21]:
# INNER JOIN query
query = """
SELECT i.product_id, i.product_name, i.quantity, l.shipment_date, l.status
FROM Inventory i
INNER JOIN Logistics l ON i.product_id = l.product_id
WHERE i.quantity > 10
ORDER BY l.shipment_date DESC;
"""

# Execute query and fetch results
df = pd.read_sql_query(query, conn)

# Display result
df

,product_id,product_name,quantity,shipment_date,status
0,3,Keyboard,50,2024-03-14,Delivered
1,2,Monitor,25,2024-03-12,In Transit
2,1,Laptop,15,2024-03-10,Shipped


In [23]:
select = pd.read_sql("""SELECT * FROM Inventory""", conn)

In [24]:
select

,product_id,product_name,quantity,price
0,1,Laptop,15,750.5
1,2,Monitor,25,200.0
2,3,Keyboard,50,30.0
3,4,Mouse,40,25.0
4,5,Tablet,10,500.0


In [25]:
select = pd.read_sql("""SELECT * FROM Logistics""", conn)

In [26]:
select

,logistics_id,product_id,shipment_date,status
0,1,1,2024-03-10,Shipped
1,2,2,2024-03-12,In Transit
2,3,3,2024-03-14,Delivered
3,4,5,2024-03-15,Pending


In [28]:
select = pd.read_sql("""SELECT * FROM Product""", conn)

In [29]:
select

,product_id,category,supplier
0,1,Electronics,TechCorp
1,2,Electronics,ScreenPro
2,3,Accessories,KeyWorld
3,4,Accessories,MouseMax
4,5,Electronics,TabMakers


In [32]:
select = pd.read_sql("""SELECT product_name, quantity FROM Inventory
                     GROUP BY product_name
                     HAVING quantity < 20""", conn)
print("Products running low on stock:")
print(select)

Products running low on stock:
  product_name  quantity
0       Laptop        15
1       Tablet        10
